<a href="https://colab.research.google.com/github/akhi07rx/Youtube-To-Google-Drive/blob/main/Youtube_To_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube

In [ ]:
from pytube import YouTube
import sys

# Define the destination folder
destination_folder = '/content/drive/MyDrive/DOWNLOADS 02/'

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

def download_youtube_video(url, destination_folder):
    try:
        youtube = YouTube(url)

        # Get the available video streams
        streams = youtube.streams.filter(type='video', progressive=True).order_by('resolution')

        # Print available video qualities
        print("\nAvailable video qualities:")
        for i, stream in enumerate(streams, start=1):
            print(f"{i}. Resolution: {stream.resolution}, Format: {stream.mime_type}")

        # Prompt the user to select the video quality
        selected_option = None
        while selected_option is None:
            user_input = input("\nEnter the number of the desired video quality (or 'exit' to go back to URL menu): ")

            if user_input.lower() == 'exit':
                return

            if user_input.lower() == 'quit':
                sys.exit("Exiting the program.")

            if user_input.isdigit() and int(user_input) in range(1, len(streams) + 1):
                # Retrieve the selected stream
                selected_stream = streams[int(user_input) - 1]
                selected_option = user_input
            else:
                print("Invalid input. Please enter a valid number, 'exit' to go back to URL menu, or 'quit' to exit the program.")

        file_name = f"{youtube.title} ({selected_stream.resolution})"
        file_path = os.path.join(destination_folder, file_name)
        selected_stream.download(output_path=destination_folder, filename=file_name)

        print(f"Video '{youtube.title}' downloaded successfully to '{file_path}'.\n")

    except Exception as e:
        print(f"An error occurred: {str(e)}")


while True:

    user_input = input("\nEnter the YouTube video URL (or 'exit' to go back to URL menu, or 'quit' to exit the program): ")

    if user_input.lower() == 'exit':
        continue

    if user_input.lower() == 'quit':
        print("Exiting the program.")
        sys.exit()

    download_youtube_video(user_input, destination_folder)
